In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/gregory_eritsyan/VESUVIUS_Challenge/jupyter notebooks
Current path:/home/gregory_eritsyan/VESUVIUS_Challenge


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.MONAI_Dict_Dataset_Module import MONAI_CSV_Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit
from monai.visualize import matshow3d

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2023-04-25 05:22:33,384 - Created a temporary directory at /tmp/tmp7x_imv6t
2023-04-25 05:22:33,386 - Writing /tmp/tmp7x_imv6t/_remote_module_non_scriptable.py


In [3]:
KAGGLE_DIR = PATH / "kaggle"

INPUT_DIR = KAGGLE_DIR / "input"

COMPETITION_DATA_DIR = INPUT_DIR / "vesuvius-challenge-ink-detection"

TRAIN_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_train_0.5.csv"
#TEST_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_test_1.csv"

In [4]:
dataset = MONAI_CSV_Scrolls_Dataset(
    z_dim=32,
    batch_size=1,
    data_csv_path=TRAIN_DATA_CSV_PATH,
    num_workers=16,
    num_samples=16,
    patch_size=(512,512),
    val_fragment_id=1,
    on_gpu=False,
    
    
)
#dataset.setup()

In [5]:

def visualize_dataloaders(dataloaders, train=True):
    i =0 
    for stage, dataloader in dataloaders.items():
        for batch_idx, batch in enumerate(dataloader):
            if i==1:
                break
            i +=1
            
            volumes = batch["volume_npy"]
            masks = batch["mask_npy"]
            
            
            if train:
                labels = batch["label_npy"]
            else: 
                labels = masks
            print(volumes.as_tensor().shape, labels.as_tensor().shape, masks.as_tensor().shape)
            for volume, mask, label in zip(volumes, masks, labels):
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                plt.suptitle(f"stage: {stage}, fragment: {batch_idx}")

                for idx, image in enumerate((volume, mask, label)):
                    matshow3d(
                        volume=image,
                        fig=axes[idx],
                        title=f"{list(image.shape)}, {image.min().item()}, {image.max().item()}",
                        vmin=0.0,
                        vmax=1.0,
                        every_n=2,
                        fill_value=1.0,
                        margin=4,
                        cmap="gray",
                    )

dataset.setup(stage="fit")

dataloaders = {
    "train": dataset.train_dataloader(),
    "val": dataset.val_dataloader(),
}

visualize_dataloaders(dataloaders)

In [6]:
lit_model = UNET_lit(
        #wandb = True,
        z_dim = 32,
        patch_size = (512,512),
        sw_batch_size=8 ,
        eta_min = 1e-5,
        t_max = 100,
        max_epochs = 3000,
        weight_decay = 0.00005,
        learning_rate = 0.0009,
        gamma = 0.85,)

#lit_model = lit_model.load_from_checkpoint('logs/Like_Kaggle/lightning_logs/version_6/checkpoints/epoch=6010-step=6011.ckpt', 
#                                           patch_size = (512,512),
#                                           learning_rate = 0.00008,
#                                           eta_min = 1e-7,
#                                          sw_batch_size = 2)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [7]:
trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=8000,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/Monai_32_2048Unet/'),
        log_every_n_steps=1,
        
        overfit_batches=0,
        precision=16,
        accumulate_grad_batches=2,)

trainer.fit(lit_model, datamodule=dataset, ckpt_path='logs/Monai_32_2048Unet/lightning_logs/version_0/checkpoints/epoch=701-step=702.ckpt')

# ckpt_path='logs/Monai_32_2048Unet/lightning_logs/version_0/checkpoints/epoch=701-step=702.ckpt'

2023-04-25 05:22:43,118 - Using 16bit native Automatic Mixed Precision (AMP)
2023-04-25 05:22:43,176 - GPU available: True (cuda), used: True
2023-04-25 05:22:43,266 - TPU available: False, using: 0 TPU cores
2023-04-25 05:22:43,268 - IPU available: False, using: 0 IPUs
2023-04-25 05:22:43,268 - HPU available: False, using: 0 HPUs
# train: 2
# val: 1
2023-04-25 05:22:43,288 - Restoring states from the checkpoint path at logs/Monai_32_2048Unet/lightning_logs/version_0/checkpoints/epoch=701-step=702.ckpt


/home/gregory_eritsyan/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:346: UserWarning: The dirpath has changed from 'logs/Monai_32_2048Unet/lightning_logs/version_0/checkpoints' to 'logs/Monai_32_2048Unet/lightning_logs/version_1/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
  warnings.warn(


2023-04-25 05:22:47,495 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2023-04-25 05:22:47,506 - 
  | Name    | Type       | Params
---------------------------------------
0 | metrics | ModuleDict | 0     
1 | model   | UNet       | 305 M 
2 | loss    | MaskedLoss | 0     
---------------------------------------
305 M     Trainable params
0         Non-trainable params
305 M     Total params
611.112   Total estimated model params size (MB)
2023-04-25 05:22:48,394 - Restored all states from the checkpoint file at logs/Monai_32_2048Unet/lightning_logs/version_0/checkpoints/epoch=701-step=702.ckpt


Sanity Checking: 0it [00:00, ?it/s]

/home/gregory_eritsyan/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/gregory_eritsyan/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [8]:
torch.save(lit_model.state_dict(), 'kaggle/original_1kbuffer_1.pth')


In [9]:
torch.cuda.empty_cache()

In [10]:
data_module = MONAI_CSV_Scrolls_Dataset(
                z_dim=16,
                batch_size=1,
                data_csv_path=TEST_DATA_CSV_PATH,
                num_workers=8,
                num_samples=16,
                patch_size=(512,512),
                val_fragment_id=1,
                stage='predict',
                on_gpu =False,
                    )

data_module.setup(stage='predict')

NameError: name 'TEST_DATA_CSV_PATH' is not defined

In [ ]:
lit_model.eval().to('cpu')
predictions = trainer.predict(lit_model.eval(), data_module)

In [ ]:
predictions

In [ ]:
def plot_image(image, title):
    fig = plt.figure()
    plt.title(title)
    plt.imshow(image, cmap="gray")
    
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    # pixels = (pixels >= thr).astype(int)
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import pandas as pd
import numpy as np
import PIL.Image as Image
import lovely_numpy as ln
submission_df = pd.read_csv("kaggle/input/vesuvius-challenge-ink-detection/sample_submission.csv")
test_df = pd.read_csv("kaggle/input/vesuvius-challenge-ink-detection/data_test_1.csv")

In [ ]:
test_df

In [ ]:
predictions_rle = []
THRESHOLD = .58
for mask_png_path, prediction in zip(test_df["mask_png"].values, predictions):
    prediction = prediction.numpy()
    mask = Image.open(mask_png_path)

    prediction_resized = np.array(Image.fromarray(prediction).resize(mask.size))
    
    prediction_masked = prediction_resized * mask
    
    prediction_thresholded = np.where(prediction_masked > THRESHOLD, 1, 0).astype(np.uint8)
    
    prediction_rle = rle(prediction_thresholded)

    predictions_rle.append(prediction_rle)

    if 1:
        #plot_image(prediction_resized, f"prediction resized {ln.lovely(prediction_resized)}")
        #plot_image(prediction_masked, f"prediction masked {ln.lovely(prediction_masked)}")
        plot_image(
            prediction_thresholded,
            f"prediction thresholded {ln.lovely(prediction_thresholded)}",
        )
    
submission_df["Predicted"] = predictions_rle
    
submission_df.to_csv("submission.csv", index=False)

submission_df

In [ ]:
#submission_df